# Bagging
## A09 
**Isabela Torres Septien Uribe**  
730667  


In [101]:
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm
from sklearn.utils import resample

Vamos a hacer 500 subset con 500000 muestras de los tres factores que me quedaron con y elegir dos factores al azar 

In [102]:
df = pd.read_csv("Default.csv")
df

,default,student,balance,income
0,No,No,729.526495,44361.625074
1,No,Yes,817.180407,12106.134700
2,No,No,1073.549164,31767.138950
3,No,No,529.250605,35704.493940
4,No,No,785.655883,38463.495880
...,...,...,...,...
9995,No,No,711.555020,52992.378910
9996,No,No,757.962918,19660.721770
9997,No,No,845.411989,58636.156980
9998,No,No,1569.009053,36669.112360


In [103]:
df['default']  = df['default'].astype('category')
df['student']  = df['student'].astype('category')
df["student"] = df["student"].map({"Yes":1, "No":0})


In [104]:
y2 = df['default'] == 'Yes'
X2 = df[['balance', "income","student"]]
X2

,balance,income,student
0,729.526495,44361.625074,0
1,817.180407,12106.134700,1
2,1073.549164,31767.138950,0
3,529.250605,35704.493940,0
4,785.655883,38463.495880,0
...,...,...,...
9995,711.555020,52992.378910,0
9996,757.962918,19660.721770,0
9997,845.411989,58636.156980,0
9998,1569.009053,36669.112360,0


In [105]:
model = LogisticRegression()
model.fit(X2, y2)
B0 = model.intercept_
B1 = model.coef_
y_pred_prob = model.predict_proba(X2)[:,1]
B0, B1

(array([-10.9018074]),
 array([[ 5.73061148e-03,  3.96161035e-06, -6.12573129e-01]]))

In [106]:
x_const = sm.add_constant(X2)
model = sm.Logit(y2, x_const).fit()
model.summary()

Optimization terminated successfully.
         Current function value: 0.078577
         Iterations 10


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                default   No. Observations:                10000
Model:                          Logit   Df Residuals:                     9996
Method:                           MLE   Df Model:                            3
Date:              mar., 25 nov. 2025   Pseudo R-squ.:                  0.4619
Time:                        17:19:45   Log-Likelihood:                -785.77
converged:                       True   LL-Null:                       -1460.3
Covariance Type:            nonrobust   LLR p-value:                3.257e-292
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        -10.8690      0.492    -22.079      0.000     -11.834      -9.904
balance        0.0057      0.000     24.737      0.000       0.005       0.006
income      3.033e-06    8.2e-06      0.370      0.712    -1.3e-05    1.91e-05
student       -0.6468      0.236     -2.738      0.006      -1.110      -0.184
==============================================================================

Possibly complete quasi-separation: A fraction 0.15 of observations can be
perfectly predicted. This might indicate that there is complete
quasi-separation. In this case some parameters will not be identified.
"""

## Bagging

In [107]:
from sklearn.tree import DecisionTreeClassifier

In [108]:
# Definimos nuestra lista de predicciones
#B0_bootstrap_m2 = []
#B1_bootstrap_m2 = []
preds_bootstrap = []

for i in range(50):

    #cols al azar
    cols = np.random.choice(X2.columns, size=2, replace=False)

    #Bootstrap
    X2_resampled, y2_resampled = resample(X2[cols], y2)

    # Modelo
    lreg_bootstrap = DecisionTreeClassifier()
    lreg_bootstrap.fit(X2_resampled, y2_resampled)

    #B0_bootstrap_m2.append(lreg_bootstrap.intercept_[0])
    #B1_bootstrap_m2.append(lreg_bootstrap.coef_[0])

    # Hacemos el agregating de forma manual 
    preds = lreg_bootstrap.predict_proba(X2[cols])[:,1]
    preds_bootstrap.append(preds)

# Convertir a matriz
preds_bootstrap = np.array(preds_bootstrap)

# Promedio y desviación
preds_mean = preds_bootstrap.mean(axis=0)
preds_std = preds_bootstrap.std(axis=0)


In [109]:
preds_mean, preds_std

(array([0.  , 0.08, 0.  , ..., 0.02, 0.02, 0.  ], shape=(10000,)),
 array([0.       , 0.2712932, 0.       , ..., 0.14     , 0.14     ,
        0.       ], shape=(10000,)))

In [111]:
#Hacemos el auc de logistic
from sklearn.metrics import roc_auc_score
auc_simple = roc_auc_score(y2, y_pred_prob)
auc_simple


0.9495714810703948

In [112]:
y2 = y2.astype(int)
y_pred_prob = (y_pred_prob >= 0.5).astype(int)

In [113]:

#hacemos el accuracy de logistic
from sklearn.metrics import accuracy_score
accuracy_simple = accuracy_score(y2, y_pred_prob)
accuracy_simple


0.9732

In [114]:
#Hacemos el auc de bagging con las preddicciones
auc = roc_auc_score(y2, preds_mean)
auc


1.0

In [115]:

#Gacemos el accuracy de bagging con las predicciones
accuracy_bagging = accuracy_score(y2, preds_mean.round())
accuracy_bagging

0.9994

## Tabla comparativa

In [116]:
tabla = pd.DataFrame({
    "Modelo": ["Logistic Regression", "Bagging"],
    "AUC": [auc_simple, auc],
    "Accuracy": [accuracy_simple, accuracy_bagging]
})
tabla

,Modelo,AUC,Accuracy
0,Logistic Regression,0.949571,0.9732
1,Bagging,1.000000,0.9994


## Conclusiones

En términos generales, tanto la regresión logística como el modelo Bagging muestran métricas muy buenas, con AUC y Accuracy superiores al 94% y 97%. Como era de esperarse se puede checar que el bagging del auc me dio un recultado perfecto en el auc y en el accuracy por lo tanto se puede ver un mejor resultado.